In [14]:
import random
import string

import numpy as np
import pandas as pd

import faker

In [15]:
products = pd.DataFrame(
    data = [
        ("Margarine", 500, "gram", "Zuivel", "Levensmiddelen", 0.99),
        ("Halfvolle melk", 1, "liter", "Zuivel", "Levensmiddelen", 1.19),
        ("Halfvolle melk", 1.5, "liter", "Zuivel", "Levensmiddelen", 1.99),
        ("Volle melk", 1, "liter", "Zuivel", "Levensmiddelen", 1.29),
        ("Magere Yoghurt", 1, "liter", "Zuivel", "Levensmiddelen", 1.19),
        ("Volle Yoghurt", 1, "liter", "Zuivel", "Levensmiddelen", 1.29),

        ("Volkoren spagheti", 500, "gram", "Graanproducten", "Levensmiddelen", 2.09),
        ("Volkoren brood", 1, "heel", "Graanproducten", "Levensmiddelen", 1.29),
        ("Volkoren brood", 1, "half", "Graanproducten", "Levensmiddelen", 0.79),
        ("Wit brood", 1, "heel", "Graanproducten", "Levensmiddelen", 0.99),
        ("Wit brood", 1, "half", "Graanproducten", "Levensmiddelen", 0.69),

        ("Witte druiven", 500, "gram", "Fruit", "Levensmiddelen", 2.79),
        ("Appels", 8, "stuks", "Fruit", "Levensmiddelen", 2.79),
        ("Peren", 4, "stuks", "Fruit", "Levensmiddelen", 2.99),
        ("Komkommer", 1, "stuks", "Groenten", "Levensmiddelen", 1.39),
        ("Tomaten", 500, "gram", "Groenten", "Levensmiddelen", 2.49),
        ("Sla krop", 1, "stuks", "Groenten", "Levensmiddelen", 1.59),
        ("Sla gesneden", 200, "gram", "Groenten", "Levensmiddelen", 1.19),

        ("Bier", 1, "krat", "Alcoholische dranken", "Levensmiddelen", 14.17),
        ("Rode wijn", 1, "liter", "Alcoholische dranken", "Levensmiddelen", 6.99),
        ("Witte wijn", 1, "liter", "Alcoholische dranken", "Levensmiddelen", 5.29),
        
        ("Shampoo", 0.3, "liter", "Haarproducten", "Persoonlijke verzorging", 6.49),
        ("Handgel", 0.3, "liter", "Hygiene producten", "Persoonlijke verzorging", 2.99),
        ("Tandpasta", 0.075, "liter", "Hygiene producten", "Persoonlijke verzorging", 2.75),
        
        
    ],
    columns=["product", "verpakking", "eenheid", "subcategorie", "categorie", "prijs"]
)

In [16]:
# Generator settings
date_range = "2023-01-01", "2023-01-31"
daily_transactions = 80, 120
transaction_items = 1, 5

In [17]:
fkr = faker.Faker("nl_NL")
transactions = []

days = pd.date_range(*date_range)
counts = np.random.uniform(*daily_transactions, len(days)).astype(int)

for day, count in zip(days, counts):
    
    for _ in range(count):
        lines = random.randint(1, 5)
        transaction = products.sample(lines)

        transaction = transaction.assign(
            datum=day,
            klant_id=fkr.unique.bothify(text='???##########', letters=string.ascii_uppercase),
            aantal= np.random.logseries(0.65, size=lines).clip(*transaction_items),
            prijs_totaal=lambda df: df["prijs"] * df["aantal"]
        )

        transactions.append(transaction)

transactions = pd.concat(transactions)

In [18]:
transactions

,product,verpakking,eenheid,subcategorie,categorie,prijs,datum,klant_id,aantal,prijs_totaal
19,Rode wijn,1.0,liter,Alcoholische dranken,Levensmiddelen,6.99,2023-01-01,VCB8431359352,1,6.99
9,Wit brood,1.0,heel,Graanproducten,Levensmiddelen,0.99,2023-01-01,VCB8431359352,3,2.97
8,Volkoren brood,1.0,half,Graanproducten,Levensmiddelen,0.79,2023-01-01,VCB8431359352,4,3.16
13,Peren,4.0,stuks,Fruit,Levensmiddelen,2.99,2023-01-01,SDF4366045731,2,5.98
3,Volle melk,1.0,liter,Zuivel,Levensmiddelen,1.29,2023-01-01,SDF4366045731,1,1.29
...,...,...,...,...,...,...,...,...,...,...
15,Tomaten,500.0,gram,Groenten,Levensmiddelen,2.49,2023-01-31,SYV6828852314,1,2.49
1,Halfvolle melk,1.0,liter,Zuivel,Levensmiddelen,1.19,2023-01-31,GBT0691657775,3,3.57
12,Appels,8.0,stuks,Fruit,Levensmiddelen,2.79,2023-01-31,VSF1357913926,1,2.79
18,Bier,1.0,krat,Alcoholische dranken,Levensmiddelen,14.17,2023-01-31,VSF1357913926,1,14.17


In [19]:
transactions.dtypes

product                 object
verpakking             float64
eenheid                 object
subcategorie            object
categorie               object
prijs                  float64
datum           datetime64[ns]
klant_id                object
aantal                   int32
prijs_totaal           float64
dtype: object

In [20]:
transactions.to_csv("transactions.csv", index=False)